In [ ]:
## Build the docker files
# docker build -t my-docker-api .
# docker run -it -p 8081:8081 my-docker-api

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import requests

# Read test set
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True)

print(testset.classes)
# ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog',
#  'frog', 'horse', 'ship', 'truck']
print(testset.data.shape)
# (10000, 32, 32, 3)

In [ ]:
# Show image with label
data_index = 1
X = testset.data[data_index]
y = testset.targets[data_index]
plt.imshow(X)
plt.show()
print(testset.classes[y])
# ship

In [ ]:
r = requests.get("http://localhost:8080/isalive")
print(r)
# <Response [200]>

In [ ]:
r = requests.post("http://localhost:8080/predict", json={
    "instances": [{ "image": X.tolist() }]
})
print(r)
# <Response [200]>

r_json = r.json()
print(r_json)
# {'predictions': ['ship']}

In [ ]:
from typing import Dict, List, Union
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if type(instances) == list else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    return response.predictions

In [ ]:
preds = predict_custom_trained_model_sample(
    project="490925619824",
    endpoint_id="812152064833486848",
    location="us-central1",
    instances={ "image": X.tolist() }
)
print(preds)
# ['ship']